# Sample Code for scraping static, historical tweets

In [ ]:
import datetime as dt
import pandas as pd
import tweepy

In [ ]:
# Credentials
ACCESS_TOKEN = "15844......"
ACCESS_SECRET = "b12vf......"
CONSUMER_KEY = "MWwT5......"
CONSUMER_SECRET = "G3qXf......"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAA......"

## Using `tweepy` ver2 to search tweets in recent 7 days
Essential access level is okay

In [ ]:
data = pd.DataFrame(columns = ["time", "text", "country"])

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
client = tweepy.Client(BEARER_TOKEN)

In [ ]:
countries = ["#argentina", "#brazil", "#england", "#france", "#portugal", "#spain"]

### Please change the date below to no earlier than 7 days from current time

In [ ]:
hashtag = "#spain (#worldcup2022 OR #fifa) lang:en"
curr_time = dt.datetime.strptime("2022-12-16 00:00:00 +0000", "%Y-%m-%d %H:%M:%S %z")
end_time = dt.datetime.strptime("2022-12-21 00:00:00 -0500", "%Y-%m-%d %H:%M:%S %z")
time_window = dt.timedelta(hours = 1)
max_tweets = 100

In [ ]:
while curr_time < end_time:
    tweets = client.search_recent_tweets(
        query = hashtag, start_time = curr_time, end_time = curr_time + time_window,
        tweet_fields = ["created_at"], max_results = max_tweets
    )

    if tweets.data is not None:
        for tweet in tweets.data:
            data.loc[len(data)] = [tweet.created_at, tweet.text, "Spain"]

    curr_time += time_window

# Use `tweepy` ver1.1 to search tweets in recent 30 days
Require elevated access

### Please change the date below to no earlier than 30 days from current time.

In [ ]:
hashtag = "#argentina (#worldcup2022 OR #fifa) lang:en"
curr_time = dt.datetime.strptime("2022-12-15 20:00:00 +0000", "%Y-%m-%d %H:%M:%S %z")
end_time = dt.datetime.strptime("2022-12-19 00:00:00 -0500", "%Y-%m-%d %H:%M:%S %z")
time_window = dt.timedelta(hours = 6)
max_tweets = 100

In [ ]:
while curr_time < end_time:
    tweet_cursor = tweepy.Cursor(
        api.search_30_day, label = "dev", query = hashtag,
        fromDate = dt.datetime.strftime(curr_time, "%Y%m%d%H%M"),
        toDate = dt.datetime.strftime(curr_time + time_window, "%Y%m%d%H%M"),
        maxResults = max_tweets
    ).items(max_tweets)

    tweet_dict = [status._json for status in tweet_cursor]
    for tweet in tweet_dict:
        timestamp = dt.datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y")

        if "extended_tweet" in tweet:
            text = tweet["extended_tweet"]["full_text"]
            data.loc[len(data)] = [timestamp, text, "Argentina"]
        elif "retweeted_status" in tweet and "extended_tweet" in tweet["retweeted_status"]:
            text = tweet["retweeted_status"]["extended_tweet"]["full_text"]
            data.loc[len(data)] = [timestamp, text, "Argentina"]

    curr_time += time_window

In [ ]:
data

In [ ]:
data.to_csv("dataset/data_static.csv")

In [ ]:
data = pd.read_csv("dataset/data_static.csv", index_col = 0)